In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from glob import glob
import warnings

warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Convolution2D, BatchNormalization, Flatten,
                                     Dropout, Dense, AveragePooling2D, Add)
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [12]:
sample_submission = pd.read_csv("./open/sample_submission.csv")

In [2]:
train_x = np.load("./npy_data/train_x_mfcc.npy", allow_pickle=True)
train_y = np.load("./npy_data/train_y_mfcc.npy", allow_pickle=True)
test_x = np.load("./npy_data/test_x_mfcc.npy", allow_pickle=True)

In [3]:
train_x.shape, train_y.shape, test_x.shape

((25520, 40, 501, 1), (25520,), (6100, 40, 501, 1))

## Model

In [4]:
def block(input_, units = 32, dropout_rate = 0.5):
    
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(input_)
    x = BatchNormalization()(x)
    x_res = x
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Add()([x, x_res])
    x = AveragePooling2D()(x)
    x = Dropout(rate=dropout_rate)(x)
    
    return x

def second_block(input_, units = 64, dropout_rate = 0.5):
    
    x = Convolution2D(units, 1, padding ="same", activation = "relu")(input_)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = Convolution2D(units * 4, 1, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x_res = x
    x = Convolution2D(units, 1, padding ="same", activation = "relu")(x)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = Convolution2D(units * 4, 1, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Convolution2D(units, 1, padding = "same", activation = "relu")(x)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = Convolution2D(units * 4, 1, padding = "same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Add()([x, x_res])
    x = AveragePooling2D()(x)
    x = Dropout(rate=dropout_rate)(x)
    
    return x

In [5]:
def build_fn():
    dropout_rate = 0.3
    
    in_ = Input(shape = (train_x.shape[1:]))
    
    block_01 = block(in_, units = 32, dropout_rate = dropout_rate)
    block_02 = block(block_01, units = 64, dropout_rate = dropout_rate)
    block_03 = block(block_02, units = 128, dropout_rate = dropout_rate)

    block_04 = second_block(block_03, units = 64, dropout_rate = dropout_rate)
    block_05 = second_block(block_04, units = 128, dropout_rate = dropout_rate)

    x = Flatten()(block_05)

    x = Dense(units = 128, activation = "relu")(x)
    x = BatchNormalization()(x)
    x_res = x
    x = Dropout(rate = dropout_rate)(x)

    x = Dense(units = 128, activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Add()([x_res, x])
    x = Dropout(rate = dropout_rate)(x)

    model_out = Dense(units = 6, activation = 'softmax')(x)
    model = Model(in_, model_out)
    return model

In [6]:
split = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 10)

pred = []
pred_ = []

for train_idx, val_idx in split.split(train_x, train_y):
    x_train, y_train = train_x[train_idx], train_y[train_idx]
    x_val, y_val = train_x[val_idx], train_y[val_idx]

    model = build_fn()
    model.compile(optimizer = keras.optimizers.Adam(0.002),
                 loss = keras.losses.SparseCategoricalCrossentropy(),
                 metrics = ['acc'])

    history = model.fit(x = x_train, y = y_train, validation_data = (x_val, y_val), epochs = 8)
    print("*******************************************************************")
    pred.append(model.predict(test_x))
    pred_.append(np.argmax(model.predict(test_x), axis = 1))
    print("*******************************************************************")

Epoch 1/8
638/638 [==============================] - 119s 131ms/step - loss: 1.8269 - acc: 0.3477 - val_loss: 1.4279 - val_acc: 0.4534
Epoch 2/8
638/638 [==============================] - 77s 121ms/step - loss: 1.2945 - acc: 0.4845 - val_loss: 1.2536 - val_acc: 0.5302
Epoch 3/8
638/638 [==============================] - 72s 112ms/step - loss: 1.2068 - acc: 0.5429 - val_loss: 1.2422 - val_acc: 0.5529
Epoch 4/8
638/638 [==============================] - 70s 110ms/step - loss: 1.0999 - acc: 0.5906 - val_loss: 1.8364 - val_acc: 0.4146
Epoch 5/8
638/638 [==============================] - 73s 114ms/step - loss: 1.0417 - acc: 0.6186 - val_loss: 1.1981 - val_acc: 0.5886
Epoch 6/8
638/638 [==============================] - 73s 115ms/step - loss: 0.9798 - acc: 0.6392 - val_loss: 2.2213 - val_acc: 0.2968
Epoch 7/8
638/638 [==============================] - 74s 116ms/step - loss: 0.9289 - acc: 0.6613 - val_loss: 2.0782 - val_acc: 0.3329
Epoch 8/8
638/638 [==============================] - 73s 115m

In [9]:
# test_ 데이터 프레임을 만들어서 나중에 sample_submission과 id를 기준으로 merge시킬 준비를 합니다.

def get_id(data):
    return np.int(data.split("\\")[1].split(".")[0])

test_ = pd.DataFrame(index = range(0, 6100), columns = ["path", "id"])
test_["path"] = glob("./open/test/*.wav")
test_["id"] = test_["path"].apply(lambda x : get_id(x))

test_.head()

,path,id
0,./open/test\1.wav,1
1,./open/test\10.wav,10
2,./open/test\100.wav,100
3,./open/test\1000.wav,1000
4,./open/test\1001.wav,1001


In [13]:
def cov_type(data):
    return np.int(data)

# 처음에 살펴본 것처럼 glob로 test data의 path는 sample_submission의 id와 같이 1,2,3,4,5.....으로 정렬 되어있지 않습니다.
# 만들어둔 test_ 데이터프레임을 이용하여 sample_submission과 predict값의 id를 맞춰줍니다.

result = pd.concat([test_, pd.DataFrame(np.mean(pred, axis = 0))], axis = 1).iloc[:, 1:]
result["id"] = result["id"].apply(lambda x : cov_type(x))

result = pd.merge(sample_submission["id"], result)
result.columns = sample_submission.columns

In [14]:
result

,id,africa,australia,canada,england,hongkong,us
0,1,0.082500,0.231429,0.166547,0.147352,0.166429,0.205743
1,2,0.136189,0.024284,0.026924,0.540895,0.044115,0.227593
2,3,0.171692,0.022535,0.024520,0.540868,0.045691,0.194694
3,4,0.130227,0.025831,0.022820,0.431846,0.159374,0.229903
4,5,0.118807,0.014341,0.014166,0.137190,0.107941,0.607555
...,...,...,...,...,...,...,...
6095,6096,0.035955,0.138235,0.104961,0.146667,0.286812,0.287371
6096,6097,0.012191,0.014799,0.025658,0.178435,0.002796,0.766120
6097,6098,0.107931,0.022889,0.022937,0.524090,0.050649,0.271504
6098,6099,0.047528,0.069540,0.071300,0.299448,0.209472,0.302712


In [15]:
result.to_csv("result/03_MFCC.csv", index = False)